In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.metrics import *
from keras import backend as K
from tensorflow.keras.applications import EfficientNetB4

In [ ]:
path = Path('../input/cassava-leaf-disease-classification/')
os.listdir(path)

In [ ]:
train = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")
train["label"] = train["label"].astype("str")
train.head()

In [ ]:
train_df, valid_df = train_test_split(train, test_size = 0.1, random_state = 1)

In [ ]:
train_df.shape, valid_df.shape

In [ ]:
train_bs = 16
valid_bs = 16
img_size = 224
target_size = (img_size, img_size)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    height_shift_range = 0.2,
    width_shift_range = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

valid_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'image_id',
    y_col = 'label',
    directory = path/'train_images',
    target_size = target_size,
    batch_size = train_bs,
    shuffle = True,
    class_mode = 'categorical'
)

validation_generator = valid_datagen.flow_from_dataframe(
    dataframe = valid_df,
    x_col = 'image_id',
    y_col = 'label',
    directory = path/'train_images',
    target_size = target_size,
    batch_size = valid_bs,
    shuffle = False,
    class_mode = 'categorical'
)


In [ ]:
METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
]

In [ ]:
#model
input_shape = (224,224,3)
efficient_net_model = EfficientNetB4(weights = None, include_top = False, input_shape = input_shape )
efficient_net_model.load_weights('../input/tfkerasefficientnetimagenetnotop/efficientnetb4_notop.h5')


for layer in efficient_net_model.layers:
    layer.trainable = True
    
model = Sequential()
model.add(efficient_net_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = METRICS)
model.summary()

In [ ]:
callbacks = [ReduceLROnPlateau(monitor = 'val_loss', patience = 2, factor = 0.5), 
            EarlyStopping(monitor = 'val_loss', patience = 2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
            ]

In [ ]:
history = model.fit(train_generator,
          epochs = 10, 
          validation_data=validation_generator,
          callbacks=callbacks, verbose = 1)

In [ ]:
from PIL import Image

In [ ]:
test_images = os.listdir('/kaggle/input/cassava-leaf-disease-classification/test_images/')

In [ ]:
predict = []

for i in test_images:
    image = Image.open(f'/kaggle/input/cassava-leaf-disease-classification/test_images/{i}')
    image = image.resize((224, 224))
    
    image = np.asarray(image)
    image = np.expand_dims(image, axis=0)
    
    predict.append(np.argmax(model.predict(image)))

In [ ]:
submission = pd.DataFrame({'image_id': test_images, 'label': predict})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=None)